In [1]:
import pandas as pd
import numpy as np
import os
from os.path import exists

path = './Data/new_humor_datasets/'
balance_path = './Data/new_humor_datasets/balanced/'
output_path = path + 'Datasets/'
datasets = ['amazon', 'yelp_reviews', 'sarcasm_headlines', 'one_liners', 'reddit_dadjokes']

In [2]:
df_amazon = pd.read_csv(path + 'amazon/data.csv')
df_yelp = pd.read_csv(path + 'yelp_reviews/data.csv')
df_headlines = pd.read_csv(path + 'sarcasm_headlines/data.csv')
df_dad_jokes = pd.read_csv(path + 'reddit_dadjokes/data.csv')
df_one_liners = pd.read_csv(path + 'one_liners/data.csv')

FileNotFoundError: [Errno 2] No such file or directory: './Data/new_humor_datasets/amazon/data.csv'

In [32]:
dfs = {'amazon': df_amazon, 'yelp': df_yelp, 'headlines': df_headlines, 'dad_jokes': df_dad_jokes, 'one_liners': df_one_liners}
for key, value in dfs.items():
    print(f'Dataset: {key}, Size = {len(value)}')

DATASET_SIZE = 19000

Dataset: amazon, Size = 19094
Dataset: yelp, Size = 19416
Dataset: headlines, Size = 28617
Dataset: dad_jokes, Size = 22048
Dataset: one_liners, Size = 32000


In [3]:
from sklearn.model_selection import train_test_split

def split_train_test_val(df, path):
    os.makedirs(path, exist_ok=True)
    train, test = train_test_split(df, test_size=0.3, shuffle=True, random_state=0)
    test, val = train_test_split(test, test_size=0.5, shuffle=True, random_state=0)
    test.to_csv(f'{path}/test.csv', index=False)
    train.to_csv(f'{path}/train.csv', index=False)
    val.to_csv(f'{path}/val.csv', index=False)

In [4]:
def process_and_save_dataset(df_new, path):
    df_new = df_new.sample(frac=1, random_state=0)
    cols = ['id', 'bert_sentence', 't5_sentence', 'target', 'label']
    df_new = df_new[cols]

    os.makedirs(path, exist_ok=True)
    df_new.to_csv(f'{path}/data.csv', index=False)

    split_train_test_val(df_new, f'{path}/with_val')

### Move amazon to temp_run dir

In [10]:
output_path_amzaon = output_path + 'amazon'
process_and_save_dataset(df_amazon, output_path_amzaon)

### Process Yelp Dataset & save

In [ ]:
df_yelp_new = pd.DataFrame()
df_yelp_new['bert_sentence'] = df_yelp['sentence']
df_yelp_new['t5_sentence'] = df_yelp['sentence']
df_yelp_new['label'] = df_yelp['label']
df_yelp_new['target'] = df_yelp_new['label'].apply(lambda label: 'funny' if label == 1 else 'not funny')
df_yelp_new = df_yelp_new[df_yelp_new['bert_sentence'].notna()]
df_yelp_new = df_yelp_new[df_yelp_new['t5_sentence'].notna()]
df_yelp_new['id'] = range(0, len(df_yelp_new))

In [12]:
output_path_yelp = output_path + 'yelp'
process_and_save_dataset(df_yelp_new, output_path_yelp)

### Process YelpShort Dataset (shorter sentences)

In [6]:
df_yelp_short_new = pd.DataFrame()
df_yelp_short_new['bert_sentence'] = df_yelp_short['sentence']
df_yelp_short_new['t5_sentence'] = df_yelp_short['sentence']
df_yelp_short_new['label'] = df_yelp_short['label']
df_yelp_short_new['target'] = df_yelp_short_new['label'].apply(lambda label: 'funny' if label == 1 else 'not funny')
df_yelp_short_new = df_yelp_short_new[df_yelp_short_new['bert_sentence'].notna()]
df_yelp_short_new = df_yelp_short_new[df_yelp_short_new['t5_sentence'].notna()]
df_yelp_short_new['id'] = range(0, len(df_yelp_short_new))

In [7]:
output_path_yelp_short = output_path + 'yelp_short'
process_and_save_dataset(df_yelp_short_new, output_path_yelp_short)

### Process Sarcasm Headlines Dataset & Save

In [13]:
df_headlines_new = pd.DataFrame()
df_headlines_new['bert_sentence'] = df_headlines['sentence']
df_headlines_new['t5_sentence'] = df_headlines['sentence']
df_headlines_new['label'] = df_headlines['label']
df_headlines_new['target'] = df_headlines_new['label'].apply(lambda label: 'funny' if label == 1 else 'not funny')
df_headlines_new = df_headlines_new[df_headlines_new['bert_sentence'].notna()]
df_headlines_new = df_headlines_new[df_headlines_new['t5_sentence'].notna()]
df_headlines_new['id'] = range(0, len(df_headlines_new))

In [14]:
output_path_headlines = output_path + 'sarcasm_headlines'
process_and_save_dataset(df_headlines_new, output_path_headlines)

### Clean Reddit Dad Jokes (as much as possible)

In [3]:
dadjokes_path = './Data/new_humor_datasets/reddit_dadjokes/'
# load Reddit Dad Jokes dataset
df_dadjokes = pd.read_csv(dadjokes_path + 'reddit_dadjokes.csv')

# clean from duplicates and reposts
df_dadjokes = df_dadjokes[df_dadjokes['joke'].apply(lambda joke: 'reposted' not in joke.lower())]
df_dadjokes.drop_duplicates(inplace=True)


In [ ]:
# df_dadjokes['id'] = range(1, len(df_dadjokes) + 1)

#### remove problematic samples

In [13]:
indices_to_remove = []

In [14]:
indices_to_remove.append(df_dadjokes[df_dadjokes.joke == 'test test'].index[0])
indices_to_remove.append(df_dadjokes[df_dadjokes['joke'].apply(lambda joke: 'Game Log Output Begins Here' in joke)].index[0])

In [15]:
df_dadjokes.drop(indices_to_remove, inplace=True)

In [17]:
df_dadjokes['id'] = range(1, len(df_dadjokes) + 1)
df_dadjokes.to_csv(dadjokes_path + 'reddit_dadjokes_with_id.csv', index=False)

# Check balanced sets

In [5]:
df_amazon = pd.read_csv(balance_path + 'amazon/data.csv')
df_yelp = pd.read_csv(balance_path + 'yelp_reviews/data.csv')
df_headlines = pd.read_csv(balance_path + 'headlines/data.csv')
df_dad_jokes = pd.read_csv(balance_path + 'dadjokes/data.csv')
df_one_liners = pd.read_csv(balance_path + 'one_liners/data.csv')

In [7]:
def print_df_labels_stat(df, df_name, sent_col_name):
    print(f'stats for {df_name}:')
    print(f'{df_name} len = {len(df)}')
    df_label_0 = df[df.label == 0]
    df_label_1 = df[df.label == 1]
    print(f'% 0 labels = {"%.2f" %  (100 * (len(df_label_0) / len(df)))}')
    print(f'% 1 labels = {"%.2f" %  (100 * (len(df_label_1) / len(df)))}')
    print(f'avg sentence length label 0 = {"%.2f" %  np.mean(df_label_0[sent_col_name].apply(lambda s: len(s)))}')
    print(f'avg sentence length label 1 = {"%.2f" %  np.mean(df_label_1[sent_col_name].apply(lambda s: len(s)))}')

In [8]:
# setting same column text name to all
df_amazon.drop('bert_sentence', axis=1, inplace=True)
df_amazon.rename(columns={'t5_sentence': 'text'}, inplace=True)
df_headlines.rename(columns={'sentence': 'text'}, inplace=True)

In [12]:
dfs = {'amazon': df_amazon, 'yelp_reviews': df_yelp, 'headlines': df_headlines, 'dadjokes': df_dad_jokes, 'one_liners': df_one_liners}
for key, value in dfs.items():
    print(f'Dataset: {key}')
    print(f'Size = {len(value)}')
    print(f'Label 0 % = {"%.2f" % (len(value[value.label == 0]) / len(value))}')
    print(f'Label 1 % = {"%.2f" % (len(value[value.label == 1]) / len(value))}')
    print_df_labels_stat(value, key, 'text')
    print('---------')
    value.to_csv(f'{balance_path}{key}/data.csv', index=False)


DATASET_SIZE = 19000

Dataset: amazon
Size = 19000
Label 0 % = 0.50
Label 1 % = 0.50
stats for amazon:
amazon len = 19000
% 0 labels = 50.00
% 1 labels = 50.00
avg sentence length label 0 = 141.13
avg sentence length label 1 = 140.47
---------
Dataset: yelp_reviews
Size = 19000
Label 0 % = 0.50
Label 1 % = 0.50
stats for yelp_reviews:
yelp_reviews len = 19000
% 0 labels = 50.00
% 1 labels = 50.00
avg sentence length label 0 = 625.78
avg sentence length label 1 = 628.60
---------
Dataset: headlines
Size = 19000
Label 0 % = 0.50
Label 1 % = 0.50
stats for headlines:
headlines len = 19000
% 0 labels = 50.00
% 1 labels = 50.00
avg sentence length label 0 = 59.58
avg sentence length label 1 = 65.32
---------
Dataset: dadjokes
Size = 19000
Label 0 % = 0.50
Label 1 % = 0.50
stats for dadjokes:
dadjokes len = 19000
% 0 labels = 50.00
% 1 labels = 50.00
avg sentence length label 0 = 86.09
avg sentence length label 1 = 86.11
---------
Dataset: one_liners
Size = 19000
Label 0 % = 0.50
Label 1 % = 0.50
stats for one_li